In [721]:
import random
import math

## BACKPROPAGATION IRIS DATASET

In [722]:
random.seed(5000)

### Carregar dados da Iris

In [723]:
with open('Dataset/data_iris.txt') as csvfile:
    csvreader = csv.reader(csvfile)
    dataset = list(csvreader)

### Padronizar dados para valores numericos

In [724]:
for row in dataset:
    row[4] = ["Iris-setosa", "Iris-versicolor", "Iris-virginica"].index(row[4])
    row[:4] = [float(row[j]) for j in range(len(row))]

### Dividir os dados para trinamento

In [725]:
random.shuffle(dataset)
datatrain = dataset[:int(len(dataset) * 0.8)]
datatest = dataset[int(len(dataset) * 0.8):]
train_X = [data[:4] for data in datatrain]
train_y = [data[4] for data in datatrain]
test_X = [data[:4] for data in datatest]
test_y = [data[4] for data in datatest]

### Multiplicação das matrizes para teste

In [726]:
def matrix_mul_bias(A, B, bias):
    C = [[0 for i in range(len(B[0]))] for i in range(len(A))]    
    for i in range(len(A)):
        for j in range(len(B[0])):
            for k in range(len(B)):
                C[i][j] += A[i][k] * B[k][j]
            C[i][j] += bias[j]
    return C

### Multiplicação do vetor A com matriz B

In [727]:
def vec_mat_bias(A, B, bias):
    C = [0 for i in range(len(B[0]))]
    for j in range(len(B[0])):
        for k in range(len(B)):
            C[j] += A[k] * B[k][j]
            C[j] += bias[j]
    return C

### Multiplicação das para Backpropagation

In [728]:
def mat_vec(A, B):
    C = [0 for i in range(len(A))]
    for i in range(len(A)):
        for j in range(len(B)):
            C[i] += A[i][j] * B[j]
    return C

### Função de Sigmoid

In [729]:
def sigmoid(A, deriv=False):
    if deriv:
        for i in range(len(A)):
            A[i] = A[i] * (1 - A[i])
    else:
        for i in range(len(A)):
            A[i] = 1 / (1 + math.exp(-A[i]))
    return A

### Parametros para treinamento

In [730]:
alfa = 0.005
epoch = 1000
neuron = [4, 5, 3]

### Iniciar pesos e bias

In [731]:
weight = [[0 for j in range(neuron[1])] for i in range(neuron[0])]
weight_2 = [[0 for j in range(neuron[2])] for i in range(neuron[1])]
bias = [0 for i in range(neuron[1])]
bias_2 = [0 for i in range(neuron[2])]

### Iniciar pesos com valores randômicos [-1.0 | 1.0]

In [732]:
for i in range(neuron[0]):
    for j in range(neuron[1]):
        weight[i][j] = 2 * random.random() - 1

for i in range(neuron[1]):
    for j in range(neuron[2]):
        weight_2[i][j] = 2 * random.random() - 1

In [733]:
show_total_cost = ''
for e in range(epoch):
    cost_total = 0

    for idx, x in enumerate(train_X):
        # forward propagation
        h_1 = vec_mat_bias(x, weight, bias)
        X_1 = sigmoid(h_1)
        h_2 = vec_mat_bias(X_1, weight_2, bias_2)
        X_2 = sigmoid(h_2)
        
        # converter para único
        target = [0, 0, 0]
        target[int(train_y[idx])] = 1

        # função de custo, erro de raiz quadrada
        eror = 0
        for i in range(neuron[2]):
            eror +=  (target[i] - X_2[i]) ** 2 
        cost_total += eror * 1 / neuron[2]

        # backpropagation
        # atualização dos pesos e bias camada 2
        delta_2 = []
        for j in range(neuron[2]):
            delta_2.append(-1 * 2. / neuron[2] * (target[j]-X_2[j]) * X_2[j] * (1-X_2[j]))

        for i in range(neuron[1]):
            for j in range(neuron[2]):
                weight_2[i][j] -= alfa * (delta_2[j] * X_1[i])
                bias_2[j] -= alfa * delta_2[j]
        
        # atualização dos pesos e bias camada 1
        delta_1 = mat_vec(weight_2, delta_2)
        for j in range(neuron[1]):
            delta_1[j] = delta_1[j] * (X_1[j] * (1-X_1[j]))
        
        for i in range(neuron[0]):
            for j in range(neuron[1]):
                weight[i][j] -=  alfa * (delta_1[j] * x[i])
                bias[j] -= alfa * delta_1[j]
    
    cost_total /= len(train_X)
    if(e % 100 == 0):
        show_total_cost += f'Custo total: {cost_total}\n'
        # print(f'Custo total: {cost_total}')

### Testes

In [734]:
res = matrix_mul_bias(test_X, weight, bias)
res_2 = matrix_mul_bias(res, weight_2, bias)

### Obter previsão

In [735]:
preds = []
for r in res_2:
    preds.append(max(enumerate(r), key=lambda x:x[1])[0])

#### Cálculo de precisão

In [736]:
acc = 0.0

for i in range(len(preds)):
    if preds[i] == int(test_y[i]):
        acc += 1

accuracy = acc / len(preds) * 100

print(show_total_cost)

print(f'Previsão: {preds}')
print(f'Precisão: {accuracy:.2f}%')

Custo total: 0.34550111123583205
Custo total: 0.13149238823242787
Custo total: 0.10949941015860527
Custo total: 0.09704320839532987
Custo total: 0.08506138069831262
Custo total: 0.07289757613509798
Custo total: 0.06139907048319877
Custo total: 0.05157342318796261
Custo total: 0.04379695079398785
Custo total: 0.03786338701243066

Previsão: [2, 0, 2, 2, 1, 1, 1, 1, 2, 1, 1, 1, 0, 2, 1, 2, 0, 1, 1, 2, 1, 1, 0, 2, 2, 0, 1, 2, 1, 1]
Precisão: 83.33%
